In [1]:
import numpy as np
import time
import concurrent
import covalent as ct
import os

rng = np.random.default_rng()

SIZE = 100_000_0
REPETITIONS = 10
PATH = "/Users/sankalpsanand/dev/covalent/covalent/data"

@ct.electron
def generate_random_list(size):
    return rng.choice(size, size=size, replace=False)

@ct.electron
def get_sorted(unsorted_list):
    return np.sort(unsorted_list)


@ct.electron
def write_to_file(file_path, data):
    with open(file_path, "w") as f:
        for num in data:
            f.write(str(num) + "\n")
    
    return file_path


def miniflow(size, path, i):
    random_list = generate_random_list(size)
    sorted_list = get_sorted(random_list)
    write_to_file(f"{path}/sorted_list_{i}.txt", sorted_list)

@ct.lattice
def workflow(size, repetitions, path):

    os.makedirs(path, exist_ok=True)

    # Generating {# repetitions} random lists
    for i in range(repetitions):
        random_list = generate_random_list(size)
        sorted_list = get_sorted(random_list)
        write_to_file(f"{path}/sorted_list_{i}.txt", sorted_list)




In [2]:
start_time = time.time()
workflow(SIZE, REPETITIONS, PATH)
print("--- %s seconds ---" % (time.time() - start_time))
!rm -rf data

--- 7.844016075134277 seconds ---


In [3]:
start_time = time.time()
# threads = []

os.makedirs(PATH, exist_ok=True)
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(REPETITIONS):
        executor.submit(miniflow, SIZE, PATH, i)

print("--- %s seconds ---" % (time.time() - start_time))
!rm -rf data

--- 6.456844091415405 seconds ---


In [4]:
result = ct.dispatch_sync(workflow)(SIZE, REPETITIONS, PATH)

print(result)
print(result.end_time - result.start_time)
!rm -rf data


Lattice Result
status: COMPLETED
result: None
inputs: {'size': 1000000, 'repetitions': 10, 'path': '/Users/sankalpsanand/dev/covalent/covalent/data'}
error: None

start_time: 2022-02-21 19:28:30.356111+00:00
end_time: 2022-02-21 19:28:43.243791+00:00

results_dir: /Users/sankalpsanand/dev/covalent/doc/source/tutorials/results
dispatch_id: cfbaa8fb-5a42-4a3c-9ed1-8f06413d3dcb

Node Outputs
------------
generate_random_list(0): [818155 277355 100520 ... 112230 573223 986071]
:parameter:1000000(1): 1000000
get_sorted(2): [     0      1      2 ... 999997 999998 999999]
write_to_file(3): /Users/sankalpsanand/dev/covalent/covalent/data/sorted_list_0.txt
:parameter:/Users/sankalpsanand/dev/covalent/covalent/data/sorted_list_0.txt(4): /Users/sankalpsanand/dev/covalent/covalent/data/sorted_list_0.txt
generate_random_list(5): [818155 277355 100520 ... 112230 573223 986071]
:parameter:1000000(6): 1000000
get_sorted(7): [     0      1      2 ... 999997 999998 999999]
write_to_file(8): /Users/sank